In [1]:
from collections import Counter
import numpy as np
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torch import nn, optim
from torch.nn.utils.rnn import pad_sequence, pack_padded_sequence, pad_packed_sequence
import matplotlib.pyplot as plt
import pandas as pd
import string
import re
from tqdm import tqdm

In [2]:
# watch nvidia-smi

In [2]:
df = pd.read_csv('habr_dataset.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4338 entries, 0 to 4337
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   url              4338 non-null   object
 1   title            4338 non-null   object
 2   theme            4338 non-null   object
 3   tags             4338 non-null   object
 4   read_time        4338 non-null   int64 
 5   number_of_views  4338 non-null   int64 
 6   date             4338 non-null   object
 7   amount_of_imgs   4338 non-null   int64 
 8   comments         4338 non-null   int64 
 9   bookmark         4338 non-null   int64 
 10  votes            4338 non-null   int64 
 11  text             4338 non-null   object
dtypes: int64(6), object(6)
memory usage: 406.8+ KB


In [3]:
def remove_punctuation(text):
    return re.sub(f"[{string.punctuation}«»—„“…\xa0]", " ", text)

In [4]:
# texts = df['text'].to_list()
# texts = [remove_punctuation(i).lower() for i in texts]
# len(texts)

4338

In [10]:
# Удаляем слишком длинные тексты
# result = []
# for i in texts:
#     if len(i) < 7000:
#         result.append(i)
# len(result)

776

In [4]:
def split_text(text, max_=4):
    return [text[i:i + max_] for i in range(0, len(text), max_)]

In [15]:
split_text(list(range(13)))

[[0, 1, 2, 3], [4, 5, 6, 7], [8, 9, 10, 11], [12]]

In [6]:
# Разьиваем длинные тексты на несколько небольших
# result = []
# for i in texts:
#     if len(i) < 2500:
#         result.append(i)
#     else:
#         result.extend(split_text(i, max_=2500))
# len(result)

28037

In [7]:
# result=result[:4000]

In [5]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [5]:
def split_sen(text):
    for i in "":
        text = text.replace('!?', '.')
    return text.split('.')

In [6]:
# Разделение по предложениям

texts = df['text'].to_list()
result = []
for i in texts:
    result.extend(split_sen(i))

texts = [remove_punctuation(i).lower() for i in result]
len(texts)

566743

In [9]:
sorted([len(i) for i in texts], reverse=True)

[6055,
 3831,
 3484,
 3391,
 3349,
 3246,
 3168,
 3140,
 2854,
 2854,
 2730,
 2600,
 2588,
 2508,
 2499,
 2472,
 2405,
 2384,
 2254,
 2189,
 2105,
 2095,
 2089,
 2087,
 2050,
 1951,
 1943,
 1896,
 1873,
 1872,
 1863,
 1825,
 1825,
 1822,
 1820,
 1818,
 1791,
 1766,
 1714,
 1712,
 1703,
 1653,
 1634,
 1627,
 1596,
 1593,
 1588,
 1581,
 1577,
 1572,
 1556,
 1544,
 1543,
 1540,
 1538,
 1537,
 1534,
 1528,
 1510,
 1487,
 1474,
 1471,
 1447,
 1438,
 1432,
 1425,
 1422,
 1416,
 1415,
 1403,
 1401,
 1390,
 1351,
 1350,
 1346,
 1342,
 1327,
 1327,
 1321,
 1320,
 1320,
 1320,
 1318,
 1316,
 1312,
 1289,
 1275,
 1271,
 1268,
 1268,
 1268,
 1263,
 1261,
 1257,
 1257,
 1255,
 1255,
 1255,
 1254,
 1250,
 1245,
 1238,
 1233,
 1228,
 1225,
 1223,
 1218,
 1218,
 1212,
 1211,
 1207,
 1203,
 1203,
 1198,
 1195,
 1194,
 1194,
 1194,
 1194,
 1194,
 1194,
 1194,
 1194,
 1189,
 1188,
 1180,
 1168,
 1168,
 1161,
 1158,
 1156,
 1155,
 1151,
 1151,
 1147,
 1141,
 1136,
 1135,
 1133,
 1132,
 1126,
 1122,
 1119,

In [10]:
Counter([len(i) for i in texts])

Counter({1: 9110,
         2: 7860,
         0: 6195,
         3: 4216,
         86: 3573,
         91: 3550,
         100: 3546,
         93: 3443,
         89: 3367,
         75: 3366,
         92: 3357,
         96: 3356,
         78: 3337,
         95: 3324,
         4: 3323,
         82: 3311,
         85: 3296,
         87: 3291,
         88: 3291,
         94: 3284,
         99: 3271,
         84: 3270,
         97: 3269,
         77: 3263,
         80: 3261,
         79: 3259,
         98: 3240,
         81: 3239,
         71: 3231,
         90: 3223,
         101: 3223,
         102: 3223,
         83: 3209,
         76: 3207,
         106: 3201,
         109: 3197,
         105: 3190,
         107: 3188,
         103: 3177,
         73: 3166,
         104: 3160,
         74: 3158,
         70: 3150,
         72: 3148,
         108: 3116,
         110: 3102,
         69: 3102,
         114: 3086,
         65: 3083,
         116: 3043,
         68: 3038,
         113: 3037,
   

In [11]:
len([i for i in texts if len(i) <= 400])

562394

In [13]:
Counter([len(i) for i in texts if len(i) <= 400])

Counter({1: 9110,
         2: 7860,
         0: 6195,
         3: 4216,
         86: 3573,
         91: 3550,
         100: 3546,
         93: 3443,
         89: 3367,
         75: 3366,
         92: 3357,
         96: 3356,
         78: 3337,
         95: 3324,
         4: 3323,
         82: 3311,
         85: 3296,
         87: 3291,
         88: 3291,
         94: 3284,
         99: 3271,
         84: 3270,
         97: 3269,
         77: 3263,
         80: 3261,
         79: 3259,
         98: 3240,
         81: 3239,
         71: 3231,
         90: 3223,
         101: 3223,
         102: 3223,
         83: 3209,
         76: 3207,
         106: 3201,
         109: 3197,
         105: 3190,
         107: 3188,
         103: 3177,
         73: 3166,
         104: 3160,
         74: 3158,
         70: 3150,
         72: 3148,
         108: 3116,
         110: 3102,
         69: 3102,
         114: 3086,
         65: 3083,
         116: 3043,
         68: 3038,
         113: 3037,
   

In [7]:
result = [i for i in texts if len(i) <= 400][:10000]

In [8]:
class Dataset(Dataset):
    def __init__(
        self,
        lines,
    ):
        self.lines = lines
        self.pad_token = '<PAD>'
        self.bos_token = '<BOS>'
        self.eos_token = '<EOS>'
        self.uniq_words = [self.pad_token, self.bos_token, self.eos_token] + self.get_uniq_words()

        self.index_to_word = {index: word for index, word in enumerate(self.uniq_words)}
        self.word_to_index = {word: index for index, word in enumerate(self.uniq_words)}

        self.pad_token_id = self.word_to_index['<PAD>']
        self.bos_token_id = self.word_to_index['<BOS>']
        self.eos_token_id = self.word_to_index['<EOS>']

        self.tokenized = [[self.word_to_index[w] for w in line.split()] for line in self.lines]

    def get_uniq_words(self):
        word_counts = Counter(word for line in self.lines for word in line.split())
        return sorted(word_counts, key=word_counts.get, reverse=True)

    def __len__(self):
        return len(self.lines)

    def __getitem__(self, index):
        return (
            torch.LongTensor([self.bos_token_id] + self.tokenized[index]),
            torch.LongTensor(self.tokenized[index] + [self.eos_token_id]),
        )

In [9]:
dataset = Dataset(result)

In [16]:
len(dataset.word_to_index), dataset.word_to_index

(406935,
 {'<PAD>': 0,
  '<BOS>': 1,
  '<EOS>': 2,
  'в': 3,
  'и': 4,
  'на': 5,
  'с': 6,
  'не': 7,
  'что': 8,
  'для': 9,
  'как': 10,
  'это': 11,
  'из': 12,
  'а': 13,
  'по': 14,
  'к': 15,
  'но': 16,
  'то': 17,
  'от': 18,
  'или': 19,
  'при': 20,
  'за': 21,
  'мы': 22,
  'так': 23,
  'если': 24,
  'у': 25,
  'его': 26,
  'о': 27,
  'до': 28,
  'он': 29,
  'их': 30,
  'можно': 31,
  'же': 32,
  'более': 33,
  'может': 34,
  'они': 35,
  'только': 36,
  'которые': 37,
  '–': 38,
  'чтобы': 39,
  'есть': 40,
  'все': 41,
  '1': 42,
  'было': 43,
  'я': 44,
  'когда': 45,
  'уже': 46,
  'будет': 47,
  '2': 48,
  'также': 49,
  'бы': 50,
  'чем': 51,
  'время': 52,
  'того': 53,
  'даже': 54,
  'году': 55,
  'во': 56,
  'был': 57,
  'том': 58,
  'очень': 59,
  'этом': 60,
  'который': 61,
  'быть': 62,
  'были': 63,
  'после': 64,
  'вы': 65,
  'этого': 66,
  'например': 67,
  'могут': 68,
  'она': 69,
  '0': 70,
  'всё': 71,
  'лет': 72,
  'года': 73,
  'где': 74,
  'больше'

In [19]:
dataset[3]

(tensor([   1, 6814, 1227,  ...,   11,    7, 9555]),
 tensor([6814, 1227,   81,  ...,    7, 9555,    2]))

In [10]:
def pad_collate(batch): # делаем тексты все одной длины
    (xx, yy) = zip(*batch)
    x_lens = [len(x) for x in xx]
    y_lens = [len(y) for y in yy]

    xx_pad = pad_sequence(xx, batch_first=True, padding_value=dataset.pad_token_id)
    yy_pad = pad_sequence(yy, batch_first=True, padding_value=dataset.pad_token_id)

    return xx_pad, yy_pad, x_lens, y_lens

In [18]:
sorted([len(i[0]) for i in dataset], reverse=True)

[23079,
 21655,
 20087,
 16516,
 15798,
 15275,
 13932,
 11922,
 11064,
 10812,
 10436,
 10291,
 10267,
 10178,
 10126,
 10066,
 9906,
 9668,
 9593,
 9547,
 9498,
 9101,
 8860,
 8669,
 8489,
 8398,
 8332,
 8256,
 8186,
 8139,
 8079,
 7950,
 7910,
 7703,
 7684,
 7674,
 7651,
 7630,
 7613,
 7604,
 7602,
 7497,
 7472,
 7455,
 7413,
 7403,
 7393,
 7368,
 7344,
 7274,
 7254,
 7241,
 7217,
 7202,
 7188,
 7127,
 7081,
 7056,
 6997,
 6991,
 6941,
 6867,
 6827,
 6717,
 6715,
 6697,
 6643,
 6580,
 6518,
 6512,
 6511,
 6472,
 6465,
 6428,
 6400,
 6400,
 6383,
 6342,
 6329,
 6305,
 6291,
 6221,
 6201,
 6197,
 6181,
 6161,
 6157,
 6098,
 6092,
 6090,
 6063,
 6062,
 6047,
 5973,
 5886,
 5885,
 5884,
 5835,
 5820,
 5817,
 5785,
 5764,
 5749,
 5741,
 5731,
 5723,
 5713,
 5701,
 5637,
 5629,
 5627,
 5621,
 5601,
 5578,
 5574,
 5557,
 5541,
 5539,
 5514,
 5496,
 5478,
 5478,
 5474,
 5449,
 5415,
 5405,
 5402,
 5387,
 5377,
 5358,
 5348,
 5334,
 5327,
 5323,
 5322,
 5316,
 5304,
 5295,
 5294,
 5272,
 526

In [36]:
pad_collate([dataset[i] for i in range(3)])

(tensor([[    1,   903, 92488,  ...,     0,     0,     0],
         [    1,     3, 12052,  ...,  4376,   148,  1295],
         [    1,   154,  4377,  ...,     0,     0,     0]]),
 tensor([[  903, 92488,   413,  ...,     0,     0,     0],
         [    3, 12052,    55,  ...,   148,  1295,     2],
         [  154,  4377,  1313,  ...,     0,     0,     0]]),
 [907, 1867, 1599],
 [907, 1867, 1599])

In [37]:
pad_collate([dataset[i] for i in range(5)])

(tensor([[    1,   903, 92488,  ...,     0,     0,     0],
         [    1,     3, 12052,  ...,     0,     0,     0],
         [    1,   154,  4377,  ...,     0,     0,     0],
         [    1,  6814,  1227,  ...,    11,     7,  9555],
         [    1,  5159,  4309,  ...,     0,     0,     0]]),
 tensor([[  903, 92488,   413,  ...,     0,     0,     0],
         [    3, 12052,    55,  ...,     0,     0,     0],
         [  154,  4377,  1313,  ...,     0,     0,     0],
         [ 6814,  1227,    81,  ...,     7,  9555,     2],
         [ 5159,  4309,    18,  ...,     0,     0,     0]]),
 [907, 1867, 1599, 4201, 853],
 [907, 1867, 1599, 4201, 853])

In [11]:
dataloader = DataLoader(dataset, batch_size=30, collate_fn=pad_collate, shuffle=True)

In [12]:
class Model(nn.Module):
    def __init__(self, vocab_len):
        super(Model, self).__init__()
        self.hidden_size = 256
        self.embedding_dim = 256
        self.num_layers = 2

        vocab_len = len(dataset.uniq_words)
        self.embedding = nn.Embedding(
            num_embeddings=vocab_len,
            embedding_dim=self.embedding_dim,
            padding_idx=0,
        )
        self.rnn = nn.GRU(
            input_size=self.embedding_dim,
            hidden_size=self.hidden_size,
            num_layers=self.num_layers,
            #dropout=0.2,
            batch_first=True,
        )
        self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(self.hidden_size, vocab_len)

    def forward(self, x, lens=None, prev_state=None):
        embed = self.embedding(x)
        if lens is None:
            output, state = self.rnn(embed, prev_state)
        else:
            embed_packed = pack_padded_sequence(embed, lens, batch_first=True, enforce_sorted=False)
            output_packed, state = self.rnn(embed_packed, prev_state)
            output, _ = pad_packed_sequence(output_packed, batch_first=True)
        output = self.drop(output)
        logits = self.fc(output)
        return logits, state

In [13]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# DEVICE = torch.device('cpu')

model = Model(len(dataset.uniq_words)).to(DEVICE).train()

In [14]:
sum(p.numel() for p in model.parameters() if p.requires_grad)

18680892

In [15]:
criterion = nn.CrossEntropyLoss(ignore_index=dataset.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-3)

In [18]:
def train(model, dataloader, criterion, optimizer, epochs):
    losses = []
    model.train()

    for epoch in range(epochs):
        for i in tqdm(dataloader):
            x, y, x_lens, y_lens = i
            optimizer.zero_grad()

            y_pred, _ = model(x.to(DEVICE), x_lens)
            loss = criterion(y_pred.transpose(1, 2), y.to(DEVICE))

            loss.backward()
            optimizer.step()
        print({ 'epoch': epoch, 'loss': loss })

        losses.append(loss.item())
        if epoch % 10 == 0:
            print({ 'epoch': epoch, 'loss': losses[-1] })

    return losses

In [19]:
loss_history = train(model, dataloader, criterion, optimizer, epochs=150)

100%|█████████████████████████████████████████| 334/334 [00:33<00:00,  9.91it/s]


{'epoch': 0, 'loss': tensor(8.7695, device='cuda:0', grad_fn=<NllLoss2DBackward0>)}
{'epoch': 0, 'loss': 8.76952075958252}


100%|█████████████████████████████████████████| 334/334 [00:32<00:00, 10.29it/s]


{'epoch': 1, 'loss': tensor(8.3415, device='cuda:0', grad_fn=<NllLoss2DBackward0>)}


100%|█████████████████████████████████████████| 334/334 [00:32<00:00, 10.21it/s]


...

{'epoch': 53, 'loss': tensor(7.9173, device='cuda:0', grad_fn=<NllLoss2DBackward0>)}


100%|█████████████████████████████████████████| 334/334 [00:32<00:00, 10.24it/s]


{'epoch': 54, 'loss': tensor(7.6083, device='cuda:0', grad_fn=<NllLoss2DBackward0>)}


 24%|█████████▉                                | 79/334 [00:07<00:25,  9.91it/s]


KeyboardInterrupt: 

In [23]:
torch.cuda.empty_cache()

In [18]:
# loss_history

In [20]:
def tokenize(value):
    return [dataset.bos_token_id]+[dataset.word_to_index[word.lower()] for word in value.split()]


def decode(token_ids):
    return ' '.join(dataset.index_to_word[token_id] for token_id in token_ids)


@torch.no_grad()
def generate(prompt, max_tokens=20):
    model.eval()
    response = []
    state = None
    prompt_tokens = tokenize(prompt)
    model_input = torch.LongTensor([prompt_tokens]).to(DEVICE)
    for _ in range(max_tokens):
        logits, state = model(model_input, prev_state=state)
        token_argmax = logits[0, -1].argmax()
        response.append(token_argmax.item())
        if response[-1] == dataset.eos_token_id:
            break
        model_input = token_argmax.view(1, 1)

    return decode(prompt_tokens + response)

In [21]:
generate('формула')

'<BOS> формула и в этом в этом в этом в том в том в том в том в том в том в'

In [22]:
@torch.no_grad()
def sample(prompt, max_tokens=20):
    model.eval()
    response = []
    state = None
    prompt_tokens = tokenize(prompt)
    model_input = torch.LongTensor([prompt_tokens]).to(DEVICE)
    for _ in range(max_tokens):
        logits, state = model(model_input, prev_state=state)
        token_probs = F.softmax(logits[0, -1], dim=-1).cpu().numpy()
        sampled_token = np.random.choice(len(token_probs), p=token_probs)
        response.append(sampled_token)
        if response[-1] == dataset.eos_token_id:
            break
        model_input = torch.LongTensor([[sampled_token]]).to(DEVICE)

    return decode(prompt_tokens + response)

In [23]:
sample('без меня')

'<BOS> без меня на околозвуковой этой по конечном баулин сгенерированных кто <EOS>'

In [26]:
sample('стать')

'<BOS> стать визуально криптографические нам создать не звезды преобразуется как минимум не взаимодействие в коробках которая <EOS>'

In [28]:
sample('исследования')

'<BOS> исследования как невозможной и различные при этому предложена частот сначала назад какие мои это вообще фотографии также нужно сестры и сделать'

In [31]:
sample('alibaba')

'<BOS> alibaba сне новая умел system вверх <EOS>'